In [386]:
import pandas as pd
import numpy as np
import itertools

In [387]:
data=pd.read_csv('aprior.csv')
data

,tid,item
0,100,"bread,Milk,tea"
1,101,"bread,Milk,pa"
2,102,"Milk,tea,zz"
3,103,"bread,Milk,tea,zz"
4,104,"bread,Milk,tea,zz"


In [388]:
min_support=50  #input 
no_of_item=int((min_support/100)*len(data))
no_of_item

2

In [389]:
#convert all tranaction element to list
column=data.columns
d=data[column[1]]
list_data=[]
for i in d:
    p=[]
    for j in range(len(i)):
        if i[j]==' ' or i[j]==',':
            continue
        p.append(i[j])
    list_data.append(p)
print(d)

def c1_matrix(data):
    unique_element=[]
    for i in data:
        for j in i:
            if j not in unique_element:
                unique_element.append(j)
    unique_element.sort()
    c1=[]
    for i in unique_element:
        c1.append([i])
    return c1

c1=c1_matrix(list_data)
print(c1)

0       bread,Milk,tea
1        bread,Milk,pa
2          Milk,tea,zz
3    bread,Milk,tea,zz
4    bread,Milk,tea,zz
Name: item, dtype: object
[['M'], ['a'], ['b'], ['d'], ['e'], ['i'], ['k'], ['l'], ['p'], ['r'], ['t'], ['z']]


In [390]:
#calculate c_i
def calculate_ci(l_i): 
    length=len(l_i[0])
    c_i=[]
    if length==1:
        for i in range(len(l_i)-1):
            for j in range(i+1,len(l_i)):
                p=l_i[i]
                q=l_i[j]
                c_i.append([p[0],q[0]])
        
        return c_i
    else:
        for i in l_i:
            union=[]
            for j in l_i:
                if i!=j:
                    p=set(i)
                    q=set(j)
                    common=p&q
                    common_len=len(common)
                    if common_len==length-1:
                        if i[0]==j[0]:
                            union=list(q.union(p))
                            p=union.sort()
                            
                            p=list(itertools.combinations(union, length))
                            
                            count=0
                            for k in p:
                                if list(k) in l_i:
                                    count+=1
                            if count==len(p):
                                if union not in c_i:
                                    c_i.append(union)
        return c_i
                                
                                      
# calculate c_i support
def ci_support(c_i,data):
    support=[]
    #print(c_i)
    for i in c_i:
        p=0
        for j in data:
            #print(i,j)
            c=set(i)&set(j)
            #print(c)
            if len(c)==len(i):
                p+=1
        support.append(p)
    return support


In [391]:
def calculate_l(c_i,c_value):
    l_matrix=[]
    l_value=[]
    for i in range(len(c_value)):
        if int(c_value[i])>=no_of_item:
            l_matrix.append(c_i[i])
            l_value.append(c_value[i])
    return l_matrix,l_value

#calculate_l(ci_matrix,support)

In [392]:
def apprior(data):
    total_li=[]
    c1=c1_matrix(data)
    c1_support=ci_support(c1,data)
    l1,l1_support=calculate_l(c1,c1_support)
    #print(l1,l1_support)
    while True:
        ci=calculate_ci(l1)
        ci_suppor=ci_support(ci,data)    
        l1,li_support=calculate_l(ci,ci_suppor)
        if len(l1)==0:
            break
        total_li.append(l1)

    return total_li

li_element=apprior(list_data)
print(li_element[-1])

[['M', 'a', 'b', 'd', 'e', 'i', 'k', 'l', 'r', 't', 'z']]


In [393]:
def subset(x):
    sub_set=[]
    p=[]
    for i in range(1,len(x)):
        p+=list(itertools.combinations(x,i))
    sub_set.append(p)
    return sub_set
    

def rule(li,confidance):
    ap_rule=[]
    for i in li:
        sub_set=subset(i)
        sub_set=sub_set[0]
        for k in range(len(sub_set)):
            j=sub_set[k]
            i_c=list(set(i)^set(j))
            support_c=ci_support([i],list_data)
            support_ci=ci_support(i_c,list_data)
            if support_c[0]/support_ci[0]>confidance:
                ap_rule.append([j,i_c])
    return ap_rule
confidance=50
rule(li_element[-1],confidance/100)

[[('r',), ['z', 'b', 'l', 'M', 'e', 'i', 'd', 't', 'k', 'a']],
 [('M', 'r'), ['z', 'b', 'l', 'e', 'i', 'd', 't', 'k', 'a']],
 [('a', 'r'), ['z', 'b', 'M', 'l', 'e', 'i', 'd', 't', 'k']],
 [('b', 'r'), ['z', 'e', 'i', 't', 'a', 'M', 'l', 'd', 'k']],
 [('d', 'r'), ['z', 'b', 'M', 'l', 'e', 'i', 't', 'k', 'a']],
 [('e', 'r'), ['z', 'b', 'M', 'l', 'i', 'd', 't', 'k', 'a']],
 [('i', 'r'), ['z', 'b', 'M', 'l', 'e', 'd', 't', 'k', 'a']],
 [('k', 'r'), ['z', 'b', 'M', 'l', 'e', 'i', 'd', 't', 'a']],
 [('l', 'r'), ['z', 'b', 'M', 'e', 'i', 'd', 't', 'k', 'a']],
 [('r', 't'), ['z', 'b', 'M', 'l', 'e', 'i', 'd', 'k', 'a']],
 [('M', 'a', 'r'), ['z', 'b', 'l', 'e', 'i', 'd', 't', 'k']],
 [('M', 'd', 'r'), ['z', 'b', 'l', 'e', 'i', 't', 'k', 'a']],
 [('M', 'e', 'r'), ['z', 'b', 'l', 'i', 'd', 't', 'k', 'a']],
 [('M', 'i', 'r'), ['z', 'b', 'l', 'e', 'd', 't', 'k', 'a']],
 [('M', 'k', 'r'), ['z', 'b', 'l', 'e', 'i', 'd', 't', 'a']],
 [('M', 'l', 'r'), ['z', 'b', 'e', 'i', 'd', 't', 'k', 'a']],
 [('M',